In [ ]:
from typing import TypedDict, Literal, get_args
import json
import random


class Creature(TypedDict):
    class Name(TypedDict):
        en: str
        vi: str
        ko: str
        jp: str

    id: str
    type: Literal["animal", "plant", "food"]
    name: Name


class Relationship(TypedDict):
    eat: list[str]
    eaten: list[str]


class Level(TypedDict):
    class Item(TypedDict):
        Direction = Literal["up", "left", "down", "right"]

        index: int
        creature_id: str
        direction: Direction

    LevelData = list[Item]

    size: int
    data: LevelData
    status: Literal["active", "done"]

In [ ]:
with open("./data/creature.json", "r", encoding="utf-8") as file:
    creatures: list[Creature] = json.load(file)

with open("./data/relationship.json", "r", encoding="utf-8") as file:
    relationship: dict[str, Relationship] = json.load(file)

directions = list(get_args(Level.Item.Direction))
creatures_dict = dict(zip([c["id"] for c in creatures], creatures))

In [ ]:
def init_level(size: int):
    data: Level.LevelData = []
    number_spots = size * size
    for index in range(number_spots):
        data.append({"index": index, "creature_id": "", "direction": ""})

    should_reset = False

    def get_items_around_item(index: int):
        res: list[Level.Item] = []
        if index - size >= 0:
            up = data[index - size]
            if up["creature_id"] == "":
                up["direction"] = "down"
            res.append(up)
        if index % size != 0 and index - 1 >= 0:
            left = data[index - 1]
            if left["creature_id"] == "":
                left["direction"] = "right"
            res.append(left)
        if index + size < number_spots:
            down = data[index + size]
            if down["creature_id"] == "":
                down["direction"] = "up"
            res.append(down)
        if (index % size) != (size - 1) and index + 1 < number_spots:
            right = data[index + 1]
            if right["creature_id"] == "":
                right["direction"] = "left"
            res.append(right)

        return res

    def get_opposite_direction(dir: Level.Item.Direction) -> Level.Item.Direction:
        if dir == "down":
            return "up"
        if dir == "right":
            return "left"
        if dir == "up":
            return "down"
        if dir == "left":
            return "right"

    def get_direction_base_on_pos(pivot: int, index: int) -> Level.Item.Direction:
        if index == pivot - size:
            return "up"
        if index == pivot - 1:
            return "left"
        if index == pivot + size:
            return "down"
        if index == pivot + 1:
            return "right"

    def set_value_for_around_items(
        index: int, steps: int = None, e: Literal["eat", "eaten"] = None
    ):
        items = get_items_around_item(index)
        items = [i for i in items if i["creature_id"] == ""]
        if not len(items):
            return

        init_item = data[index]
        init_creature = creatures_dict[init_item["creature_id"]]

        if steps is None:
            steps = random.choice([1, 2, 3])

        if e is None:
            e = random.choice(["eat", "eaten"])
            if not len(relationship[init_creature["id"]]["eat"]):
                e = "eaten"
            elif not len(relationship[init_creature["id"]]["eaten"]):
                e = "eat"

        if e == "eaten":
            items = random.sample(items, k=random.randint(1, len(items)))
            creatures_eat = relationship[init_item["creature_id"]]["eaten"]
            if not len(creatures_eat):
                return
            for item in items:
                item["creature_id"] = random.choice(creatures_eat)

            init_item["direction"] = ""

            next_step = steps - 1
            if next_step > 0:
                for item in items:
                    set_value_for_around_items(item["index"], next_step, "eaten")

        elif e == "eat":
            item = random.choice(items)
            init_item["direction"] = get_opposite_direction(item["direction"])
            creatures_eaten = relationship[init_item["creature_id"]]["eat"]
            if not len(creatures_eaten):
                init_item["creature_id"] = ""
                init_item["direction"] = ""
                return
            item["creature_id"] = random.choice(creatures_eaten)
            item["direction"] = ""

            next_step = steps - 1
            if next_step > 0:
                set_value_for_around_items(item["index"], next_step, "eaten")

    def set_item_base_on_around_items(index: int):
        items = get_items_around_item(index)
        items = [
            i
            for i in items
            if (i["creature_id"] != "" and len(relationship[i["creature_id"]]["eaten"]))
        ]

        if not len(items):
            nonlocal should_reset
            should_reset = True
            return

        item = random.choice(items)
        creatures_eat = relationship[item["creature_id"]]["eaten"]
        data[index] = {
            "index": index,
            "creature_id": random.choice(creatures_eat),
            "direction": get_direction_base_on_pos(pivot=index, index=item["index"]),
        }
        return

    def set_value_at_index(index: int):
        items = get_items_around_item(index)
        items_can_eat = [
            i
            for i in items
            if (i["creature_id"] != "" and len(relationship[i["creature_id"]]["eaten"]))
        ]
        items_empty = [i for i in items if i["creature_id"] == ""]

        is_set_around = False
        if len(items_can_eat) and len(items_empty):
            is_set_around = random.choice([True, False])
        if not len(items_can_eat):
            is_set_around = True

        if is_set_around:
            init_creature = random.choice(creatures)
            data[index] = {
                "index": index,
                "creature_id": init_creature["id"],
                "direction": random.choice(directions),
            }
            set_value_for_around_items(index)
        else:
            set_item_base_on_around_items(index)

    init_index = random.randint(0, number_spots - 1)
    init_creature = random.choice(creatures)
    data[init_index] = {
        "index": init_index,
        "creature_id": init_creature["id"],
        "direction": random.choice(directions),
    }
    set_value_for_around_items(init_index)

    while True:
        if should_reset:
            break
        empty_spots = [d for d in data if d["creature_id"] == ""]
        if not len(empty_spots):
            break
        set_value_at_index(random.choice(empty_spots)["index"])

    if should_reset:
        return init_level(size)

    level: Level = {"size": size, "data": data, "status": "active"}

    return level

In [222]:
levels = []

for i in range(20):
    difficult = random.randint(3, 7)
    level = init_level(difficult)
    levels.append(level)

with open("./data/levels.json", "w", encoding="utf-8") as file:
    json.dump(levels, file, indent=2)